In [12]:
import pandas as pd
import os
import re
import json
import time
import requests
import random
from bs4 import BeautifulSoup
from collections import Counter

In [13]:
street_articles = json.load(open('../data/articles_34st.json'))

In [14]:
street_articles[2]

{'id': 45636,
 'type': 'article',
 'srn': 'c16a2638-f85e-496b-b592-f12693b4232f',
 'title': 'A Conversation on ‘Megalopolis’',
 'slug': 'megalopolis-francis-ford-coppola-adam-driver-aubrey-plaza-flop-genius',
 'content': 'Jackson: Francis Ford Coppola, are you okay?  To say that I saw Megalopolis when I was abroad at the Cannes Film Festival is a fraction of the truth. What actually happened was that after an all–nighter waiting for another movie, my friends and I got in line at 4:00 a.m. for the 8:30 a.m. Megalopolis showing. Our efforts paid off; we were some of the first people in the theater from the last–minute line. I paid the price for my passion, though, when I fell asleep approximately five minutes into the movie, jolting awake at various times over the next two–plus hours to catch a random disjointment of scenes. The film seemed like a confusing, ridiculous mess.  Back in the US, my non–sleep–deprived review of the film in its entirety is about the same. Megalopolis is cramme

In [28]:
section_urls = {
    "features": "https://www.34st.com/section/features?page=1&per_page=500",
    "focus": "https://www.34st.com/section/focus?page=1&per_page=500",
    "ego": "https://www.34st.com/section/ego?page=1&per_page=500",
    "word_on_the_street": "https://www.34st.com/section/word-on-the-street?page=1&per_page=500",
    "film": "https://www.34st.com/section/film?page=1&per_page=500",
    "music": "https://www.34st.com/section/music?page=1&per_page=500",
    "arts": "https://www.34st.com/section/arts?page=1&per_page=500",
    "style": "https://www.34st.com/section/style?page=1&per_page=500",
}


In [29]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
# Fetch HTML for each section URL
section_html = {}
for section, url in section_urls.items():
     try:
        response = requests.get(url, headers=headers)
        print(f"Fetching {url} returned status code {response.status_code}")
        if response.status_code == 200:
            print(f"Successfully fetched {url}")
            section_html[section] = response.text
            # Parse the HTML content
            section_html[section + "_soup"] = BeautifulSoup(response.text, 'html.parser')
            print(f"Successfully fetched {url}")
        else:
            print(f"Failed to fetch {url}, status code: {response.status_code}")
     except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")

Fetching https://www.34st.com/section/features?page=1&per_page=500 returned status code 200
Successfully fetched https://www.34st.com/section/features?page=1&per_page=500
Successfully fetched https://www.34st.com/section/features?page=1&per_page=500
Fetching https://www.34st.com/section/focus?page=1&per_page=500 returned status code 200
Successfully fetched https://www.34st.com/section/focus?page=1&per_page=500
Successfully fetched https://www.34st.com/section/focus?page=1&per_page=500
Fetching https://www.34st.com/section/ego?page=1&per_page=500 returned status code 200
Successfully fetched https://www.34st.com/section/ego?page=1&per_page=500
Successfully fetched https://www.34st.com/section/ego?page=1&per_page=500
Fetching https://www.34st.com/section/word-on-the-street?page=1&per_page=500 returned status code 200
Successfully fetched https://www.34st.com/section/word-on-the-street?page=1&per_page=500
Successfully fetched https://www.34st.com/section/word-on-the-street?page=1&per_pag

In [19]:
section_urls

{'features': 'https://www.34st.com/section/features?page=1&per_page=500',
 'focus': 'https://www.34st.com/section/focus?page=1&per_page=500',
 'ego': 'https://www.34st.com/section/ego?page=1&per_page=500',
 'word_on_the_street': 'https://www.34st.com/section/word-on-the-street?page=1&per_page=500',
 'film': 'https://www.34st.com/section/film?page=1&per_page=500',
 'music': 'https://www.34st.com/section/music?page=1&per_page=500',
 'arts': 'https://www.34st.com/section/arts?page=1&per_page=500',
 'style': 'https://www.34st.com/section/style?page=1&per_page=500'}

In [30]:
for article in street_articles:
    article_url = article["url"]
    article["section"] = None  # Default to None if no section is found
    
    # Loop through each section in section_html
    for section, html in section_html.items():
        # Get the parsed HTML for the current section
        soup = section_html.get(section + "_soup", None)  # Using '_soup' to get parsed HTML
        if soup is None:
            continue  # If no parsed HTML (soup), skip this section
        
        # Check if article_url is present in the section (you can customize this to look for specific elements)
        # For example, looking for links that match the article_url
        links_in_section = soup.find_all("a", href=True)  # Find all links in the section
        
        # Check if the article URL is in any of the links
        for link in links_in_section:
            if article_url in link["href"]:
                article["section"] = section  # Assign the section if URL is found
                print(f"Article URL {article_url} matched section: {section}")  # Print for verification
                break  # Stop checking links if we found a match
        
        if article["section"]:
            break  # Stop checking other sections once a match is found

Article URL /article/2024/11/crows-penn-records-good-company-msd-almost-vegan-kapacity matched section: music
Article URL /article/2024/11/philadelphia-school-teachers-pay-underfunding-extended matched section: focus
Article URL /article/2024/11/megalopolis-francis-ford-coppola-adam-driver-aubrey-plaza-flop-genius matched section: film
Article URL /article/2024/11/speak-no-evil-mackenzie-davis-interview matched section: film
Article URL /article/2024/11/questlove-philadephia-community-philanthropy-roots-september matched section: music
Article URL /article/2024/11/charli-brat-remix-rewind-experimental-features matched section: music
Article URL /article/2024/11/mask-and-wig-comedy-show-troupe-legally-bond-penn matched section: arts
Article URL /article/2024/11/bike-lane-infrastructure-get-out-the-bike-lane-bill-philadelphia-mayor-parker-philly matched section: features
Article URL /article/2024/10/horror-plays-halloween-playright-hamilton-philadelphia matched section: arts
Article URL 

In [32]:
with open('../data/articles_34st.json', 'w', encoding='utf-8') as out:
    out.write(json.dumps(street_articles, indent=4))